In [143]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

In [40]:
look_events=["CHAMPION_SPECIAL_KILL","CHAMPION_KILL","ELITE_MONSTER_KILL","BUILDING_KILL"] 

minute = 15

folder_path = "data/IRON-Timeline" #Change to your folder path


In [94]:
#PARTICIPANT FRAMES

def part_frames(json_file):
    
    data = pd.DataFrame(json_file["info"]["frames"][minute]["participantFrames"])
    
    data = data.transpose().drop(columns=["currentGold", "goldPerSecond", "participantId", "position", "timeEnemySpentControlled", "level"])
    data.reset_index(drop=True, inplace=True)

    damage_stats = pd.json_normalize(data["damageStats"])
    champion_stats = pd.json_normalize(data["championStats"])

    pf_all = pd.concat([data, damage_stats, champion_stats], axis=1).drop(columns=["championStats","damageStats"])
    pf_all = pf_all.astype(int)

    conditions = [
            (pf_all.index >= 0) & (pf_all.index <= 4),
            (pf_all.index >= 5) & (pf_all.index <= 9)
        ]
    values = [100, 200]

    pf_all['team'] = np.select(conditions, values, default=0)

    pf_all = pf_all.groupby("team").sum().reset_index()

    return pf_all


#EVENTS & TARGET

def find_indices(data):
    monster_indices = []
    for index, item in enumerate(data):
        if item.get('type') in look_events:
            monster_indices.append(index)
    return monster_indices

def get_events(json_file):
    elements_filtered = []
    for i in range(minute + 1):
        try:
            events = json_file["info"]["frames"][i]["events"]
        except IndexError:
            break
        monster_indices = find_indices(events)
        if monster_indices:
            new_elements = [events[e] for e in monster_indices]
            elements_filtered.extend(new_elements)

    events_filtered = pd.DataFrame(elements_filtered)
    return events_filtered


def transform_events(json_file):
    required_columns = ["killerId", "type", "killType", "monsterSubType", "monsterType", "towerType"]

    events = get_events(json_file)

    missing_columns = [col for col in required_columns if col not in events.columns]

    for column in missing_columns:
        events[column] = np.nan

    events = events[required_columns]
    events = events[events["killerId"] != 0]

    events.loc[events["monsterType"] == "DRAGON", "monsterType"] = events.loc[events["monsterType"] == "DRAGON", "monsterSubType"]

    events = events.drop(columns=["monsterSubType"])

    events["kills"] = (events["type"] == "CHAMPION_KILL").astype(int)

    one_hot_encoded = pd.get_dummies(events[['killType', "monsterType", "towerType"]])

    events_encoded = pd.concat([events[['killerId', "kills"]], one_hot_encoded], axis=1)
    
    if 'killType' in events_encoded.columns:
        events_encoded.drop(columns=['killType'], inplace=True)
    if 'monsterType' in events_encoded.columns:
        events_encoded.drop(columns=['monsterType'], inplace=True)
    if 'towerType' in events_encoded.columns:
        events_encoded.drop(columns=['towerType'], inplace=True)
    
    events_encoded = events_encoded.groupby("killerId").sum().reset_index()

    events_encoded["killerId"] = events_encoded["killerId"] - 1

    events_encoded = events_encoded.groupby("killerId").sum().reset_index()

    conditions = [
        (events_encoded['killerId'] >= 0) & (events_encoded['killerId'] <= 4),
        (events_encoded['killerId'] >= 5) & (events_encoded['killerId'] <= 9)
    ]
    values = [100, 200]

    events_encoded['team'] = np.select(conditions, values, default=0)

    events_encoded = events_encoded.groupby("team").sum().reset_index().drop(columns=["killerId"])
    
    return events_encoded


def merge_dfs(json_file):
    events = transform_events(json_file)
    frames = part_frames(json_file)
    
    dfs = frames.merge(events,how="left",on="team")
    
    dfs["matchId"] = json_file["metadata"]["matchId"]
    
    last_event = json_file["info"]["frames"][-1]["events"][-1]
    dfs["target"] = dfs["team"].apply(lambda x: 1 if x == last_event.get("winningTeam") else 0)
    
    dfs.drop(columns=["team"],inplace=True)
    
    return dfs



In [95]:
#LOOP PARA TODOS LOS JSON


all_events = None


for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        json_file = pd.read_json(file_path)
           
        try:             
            if all_events is None:
                all_events = merge_dfs(json_file)
            else:
                all_events = pd.concat([all_events, merge_dfs(json_file)], ignore_index=True)
        
        except IndexError:
            continue
     
columns_to_convert = all_events.columns[~all_events.columns.isin(['matchId',"target"])]   
    
all_events[columns_to_convert] = all_events[columns_to_convert].fillna(0).astype(int) 

In [96]:
#DIFERENCIAS ENTRE PARTIDAS


# Seleccionar todas las columnas excepto "matchId y"
columnas_no_matchId = all_events.columns[~ all_events.columns.isin(["matchId", "target"])]

# Seleccionar solo las columnas numéricas
columnas_numericas = all_events.select_dtypes(include=[np.number]).drop(columns = "target")

# Crear un nuevo DataFrame vacío con la misma columna "matchId, target" y las columnas numéricas
all_events_diff = pd.DataFrame(columns=["matchId", "target"] + list(columnas_numericas.columns))

# Obtener el número total de filas en df_trabajo
total_filas = columnas_numericas.shape[0]

# Iterar a través de las filas de columnas_numericas
for i in range(0, total_filas - 1, 2):
    # Obtener las dos filas a restar
    fila1 = columnas_numericas.iloc[i]
    fila2 = columnas_numericas.iloc[i+1]
    
    # Calcular la resta de las dos filas
    resta_filas = fila1 - fila2
    
    # Agregar la resta al nuevo DataFrame df_partida_diff
    all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)

# Asignar el primer valor de las columnas booleanas en cada fila del nuevo DataFrame
for columna in all_events.select_dtypes(include=[bool]):
    all_events_diff[columna] = all_events[columna].iloc[::2].reset_index(drop=True)

# Asignar el primer valor de la columna "matchId" en cada fila del nuevo DataFrame
all_events_diff["matchId"] = all_events["matchId"].iloc[::2].reset_index(drop=True)

# Asignar el primer valor de la columna "target" en cada fila del nuevo DataFrame
all_events_diff["target"] = all_events["target"].iloc[::2].reset_index(drop=True)

/tmp/ipykernel_7250/3593382592.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)
/tmp/ipykernel_7250/3593382592.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)
/tmp/ipykernel_7250/3593382592.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)
/tmp/ipykernel_7250/3593382592.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)
/tmp/ipykernel_7

In [129]:
pd.set_option('display.max_columns', None)


all_df = all_events_diff.drop(columns="matchId")
all_df

,target,jungleMinionsKilled,minionsKilled,totalGold,xp,magicDamageDone,magicDamageDoneToChampions,magicDamageTaken,physicalDamageDone,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,abilityHaste,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,bonusArmorPenPercent,bonusMagicPenPercent,ccReduction,cooldownReduction,health,healthMax,healthRegen,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,physicalVamp,power,powerMax,powerRegen,spellVamp,kills,killType_KILL_FIRST_BLOOD,killType_KILL_MULTI,monsterType_CHEMTECH_DRAGON,monsterType_FIRE_DRAGON,monsterType_RIFTHERALD,towerType_OUTER_TURRET,monsterType_WATER_DRAGON,monsterType_HEXTECH_DRAGON,monsterType_AIR_DRAGON,monsterType_EARTH_DRAGON,towerType_INNER_TURRET,killType_KILL_ACE,towerType_BASE_TURRET,towerType_NEXUS_TURRET
0,0,-18,-28,-10498,-1398,20946,183,-1189,-101528,-12708,13345,-43110,-15219,14849,37472,-2693,2692,0,173,16,0,0,-234,-152,0,0,-25,0,153,386,-87,-9,13,0,10,-52,-2,0,1242,172,-98,0,-14,-1,0,-1,1,1,-1,0,0,0,0,0,0,0,0
1,0,50,-122,-3927,-2769,42767,4011,-3530,-72832,-10754,8307,-24172,-6669,4707,5891,70,-70,0,66,57,0,0,-226,-50,0,0,-36,0,223,231,84,0,0,7,-31,-78,-12,0,434,495,-262,0,-4,1,0,0,0,0,-1,1,0,0,0,0,0,0,0
2,0,-8,-2,-2589,-1853,-30518,-8799,10680,32268,10683,-15131,4653,464,-3028,2901,-1422,1422,0,-80,67,0,0,8,-133,0,0,10,0,-431,-507,77,3,0,0,-21,-247,2,0,1172,692,73,0,-6,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0
3,0,16,14,-75,630,-9309,-5163,4390,31555,-878,-650,26715,-6660,4360,4470,-620,619,0,-35,-7,0,0,14,275,0,0,-11,0,-644,-17,46,6,-18,0,16,-124,-2,0,-221,-2135,-62,0,-3,-1,0,1,0,-1,0,-1,0,0,0,0,0,0,0
4,1,12,-13,-467,-1120,-21851,1381,-1924,2448,-4739,5033,-18273,-2068,1819,1130,1290,-1289,0,157,-68,0,0,-108,59,0,0,-23,0,-1266,-1027,118,13,36,-7,-41,-172,-6,0,9612,9962,-87,0,-8,1,0,0,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,0,-28,-89,-2889,-2247,-39163,334,-1333,2494,2714,-3465,-52161,490,-2238,-15488,-2558,2558,0,-42,262,0,0,-52,-45,0,0,11,0,-1041,-773,-9,-6,-12,0,51,190,-2,0,-348,-1135,210,0,0,1,0,0,0,-1,-1,0,0,0,1,0,0,0,0
4701,0,9,92,2054,3487,44388,3683,-3875,1957,-5896,10821,48413,-4332,9065,2068,-2118,2118,0,-80,21,0,0,-34,116,0,0,5,0,-415,1000,-28,5,-18,10,13,-90,5,0,575,-653,85,0,1,-1,-1,0,-1,1,0,1,0,0,0,0,0,0,0
4702,0,-20,-6,-4489,-3432,-34277,-7212,7402,4399,-305,-1388,-44121,-7881,6378,-14240,-363,362,0,31,-49,0,0,-74,186,0,0,-7,0,-262,-1111,-31,8,-24,0,34,28,0,0,-1116,-2512,-129,0,-4,1,-1,-1,0,0,-1,0,0,0,0,0,0,0,0
4703,0,-20,29,-1385,-1197,-14360,-6190,5773,18758,3582,-2340,15056,-1879,2704,10661,729,-729,0,-7,-110,0,0,-29,188,0,0,-52,0,-1249,-122,-3,0,-30,-10,-40,-132,-8,0,-190,-1376,-168,0,2,1,1,0,-1,-1,0,0,0,-1,0,0,0,0,0


In [150]:
all_events_diff[all_events_diff["matchId"] == "LA2_1293781450"]

,matchId,target,jungleMinionsKilled,minionsKilled,totalGold,xp,magicDamageDone,magicDamageDoneToChampions,magicDamageTaken,physicalDamageDone,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,abilityHaste,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,bonusArmorPenPercent,bonusMagicPenPercent,ccReduction,cooldownReduction,health,healthMax,healthRegen,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,physicalVamp,power,powerMax,powerRegen,spellVamp,kills,killType_KILL_FIRST_BLOOD,killType_KILL_MULTI,monsterType_CHEMTECH_DRAGON,monsterType_FIRE_DRAGON,monsterType_RIFTHERALD,towerType_OUTER_TURRET,monsterType_WATER_DRAGON,monsterType_HEXTECH_DRAGON,monsterType_AIR_DRAGON,monsterType_EARTH_DRAGON,towerType_INNER_TURRET,killType_KILL_ACE,towerType_BASE_TURRET,towerType_NEXUS_TURRET
1563,LA2_1293781450,0,15,-81,-8131,-2271,-9467,-246,940,-21784,-5394,1372,-46290,-6220,2893,-15042,-581,581,0,-17,-51,0,0,-52,-68,0,0,2,0,-2442,-828,-2,-4,12,0,-58,-25,-2,0,-600,-1475,-185,0,-11,-1,-1,0,0,-1,-1,0,0,-1,0,0,0,0,0


In [139]:
def duplicates(df):
    print(f"Duplicates droped: {df.duplicated().sum()}")

    df = df.drop_duplicates()

    return df

def scale(df):
    # Standard or Robust if there are many outliers
    scaler = RobustScaler()
    
    scaled_array = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_array, columns=df.columns)
    
    return scaled_df

In [140]:
def preprop(df):
    
    df = duplicates(df)
    
    df = scale(df)
    
    return pd.DataFrame(df)

In [145]:
X = all_df.drop(columns="target")
y =all_df["target"]


X_preprop = preprop(X)

Duplicates droped: 0


In [146]:
X_preprop

,jungleMinionsKilled,minionsKilled,totalGold,xp,magicDamageDone,magicDamageDoneToChampions,magicDamageTaken,physicalDamageDone,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,abilityHaste,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,bonusArmorPenPercent,bonusMagicPenPercent,ccReduction,cooldownReduction,health,healthMax,healthRegen,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,physicalVamp,power,powerMax,powerRegen,spellVamp,kills,killType_KILL_FIRST_BLOOD,killType_KILL_MULTI,monsterType_CHEMTECH_DRAGON,monsterType_FIRE_DRAGON,monsterType_RIFTHERALD,towerType_OUTER_TURRET,monsterType_WATER_DRAGON,monsterType_HEXTECH_DRAGON,monsterType_AIR_DRAGON,monsterType_EARTH_DRAGON,towerType_INNER_TURRET,killType_KILL_ACE,towerType_BASE_TURRET,towerType_NEXUS_TURRET
0,-0.642857,-0.371429,-1.772277,-0.326865,0.422675,0.015519,-0.102414,-1.639410,-1.560770,1.734284,-0.708725,-1.752318,1.898248,2.563785,-1.877762,1.852103,0.0,0.851485,0.200000,0.0,0.0,-1.529412,-1.062937,0.0,0.0,-0.568182,0.0,0.059271,0.371269,-0.635714,-0.750000,0.361111,0.0,0.185185,-0.233184,-0.50,0.0,0.690476,0.083820,-0.541667,0.0,-1.555556,-1.0,0.0,-1.0,1.0,0.5,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.785714,-1.714286,-0.669575,-0.647416,0.866588,0.487006,-0.387832,-1.173286,-1.322565,1.081270,-0.388502,-0.785343,0.638216,0.355015,0.030387,-0.021710,0.0,0.321782,0.682353,0.0,0.0,-1.477124,-0.349650,0.0,0.0,-0.818182,0.0,0.094732,0.226679,0.585714,0.000000,0.000000,7.0,-0.574074,-0.349776,-3.00,0.0,0.243079,0.255172,-1.517857,0.0,-0.444444,0.0,0.0,0.0,0.0,0.0,-0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.285714,0.000000,-0.445041,-0.433248,-0.624278,-1.090775,1.344672,0.533908,1.290747,-1.956708,0.098901,0.021375,-0.322773,0.145895,-1.000000,0.990502,0.0,-0.400990,0.800000,0.0,0.0,0.052288,-0.930070,0.0,0.0,0.227273,0.0,-0.236575,-0.461754,0.535714,0.250000,0.000000,0.0,-0.388889,-1.107623,0.50,0.0,0.651717,0.359682,0.476190,0.0,-0.666667,0.0,0.0,0.0,0.0,-0.5,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.571429,0.228571,-0.023158,0.147300,-0.192815,-0.642936,0.577786,0.522327,-0.118615,-0.079715,0.471948,-0.784325,0.595105,0.255630,-0.446133,0.445726,0.0,-0.178218,-0.070588,0.0,0.0,0.091503,1.923077,0.0,0.0,-0.250000,0.0,-0.344478,-0.004664,0.314286,0.500000,-0.500000,0.0,0.296296,-0.556054,-0.50,0.0,-0.119601,-1.140053,-0.327381,0.0,-0.333333,-1.0,0.0,1.0,0.0,-0.5,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.428571,-0.157143,-0.088941,-0.261866,-0.447962,0.163074,-0.192026,0.049527,-0.589297,0.656902,-0.288755,-0.264985,0.279414,0.022031,0.872928,-0.848711,0.0,0.772277,-0.788235,0.0,0.0,-0.705882,0.412587,0.0,0.0,-0.522727,0.0,-0.659574,-0.946828,0.828571,1.083333,1.000000,-7.0,-0.759259,-0.771300,-1.50,0.0,5.325028,5.277454,-0.476190,0.0,-0.888889,0.0,0.0,0.0,1.0,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,-1.000000,-1.242857,-0.495385,-0.525368,-0.800146,0.034118,-0.119971,0.050274,0.319273,-0.444588,-0.861769,0.024316,-0.224624,-1.140229,-1.784530,1.761194,0.0,-0.212871,3.094118,0.0,0.0,-0.339869,-0.314685,0.0,0.0,0.250000,0.0,-0.545593,-0.709888,-0.078571,-0.500000,-0.333333,0.0,0.944444,0.852018,-0.50,0.0,-0.189922,-0.609549,1.291667,0.0,0.000000,0.0,0.0,0.0,0.0,-0.5,-0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4701,0.321429,1.342857,0.334116,0.815291,0.899565,0.446607,-0.429895,0.041551,-0.730343,1.407129,0.838840,-0.521036,1.179650,0.087635,-1.480663,1.462687,0.0,-0.400990,0.258824,0.0,0.0,-0.222222,0.811189,0.0,0.0,0.113636,0.0,-0.228470,0.944030,-0.214286,0.416667,-0.500000,10.0,0.240741,-0.403587,1.25,0.0,0.321152,-0.353846,0.547619,0.0,0.111111,-1.0,-0.5,0.0,-1.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0

In [149]:
# Instantiate model
log_reg = LogisticRegression(max_iter=10000)

# Scoring on multiple folds aka Cross Validation
scores = cross_val_score(log_reg, X_preprop, y, cv=10)
print(f"Score:{scores.mean()}")

# Fit model
log_model = LogisticRegression().fit(X_preprop, y)

# Performs Permutation
permutation_score = permutation_importance(log_model, X_preprop, y, n_repeats=10)

# Unstack results showing the decrease in performance after shuffling features
importance_df = pd.DataFrame(np.vstack((X_preprop.columns,
                                        permutation_score.importances_mean)).T)
importance_df.columns=['feature','score decrease']

# Show the important features
importance_df.sort_values(by="score decrease", ascending = False)

Score:0.7817233590820799


/home/santiagop/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,feature,score decrease
2,totalGold,0.12831
38,powerMax,0.025122
37,power,0.019065
51,monsterType_EARTH_DRAGON,0.005845
45,monsterType_FIRE_DRAGON,0.004718
1,minionsKilled,0.004485
6,magicDamageTaken,0.004208
50,monsterType_AIR_DRAGON,0.003528
48,monsterType_WATER_DRAGON,0.003082
3,xp,0.002678


In [153]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression

#Split the dataset in train / test
X_train, X_test, y_train, y_test = train_test_split(X_preprop, y, test_size=0.3, random_state=88)

log_reg = LogisticRegression(max_iter=10000)

log_model = LogisticRegression().fit(X_train, y_train)



/home/santiagop/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [154]:
scores = cross_val_score(log_reg, X_train, y_train, cv=10)
print(f"Score:{scores.mean()}")

Score:0.775582573454914


In [155]:
train_score = log_model.score(X_train, y_train)
print("Train Accuracy:", train_score)

Train Accuracy: 0.783480109322806


In [156]:
#Evaluar el modelo en los datos de prueba: 
test_score = log_model.score(X_test, y_test)
print("Test Accuracy:", test_score)

Test Accuracy: 0.7875354107648725


In [158]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

y_pred = log_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7875354107648725
